<a href="https://colab.research.google.com/github/BrunoBustos96/DeteccionSomnolenciaCNN/blob/main/Parte4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Desarrollo de la aplicación de la cámara web**

Este notebook contiene los diferentes pasos para desarrollar la aplicación de la cámara web.

El usuario necesita instalar:

* homebrew (Un sistema de gestión de paquetes)
* cmake: utilizar `brew install cmake` de la linea de comandos despues de la instalacion homebrew
* dlib: puede ser instalado con `pip install dlib` despues de que cmake está listo
* face_recognition: despues de haber instalado todo lo anterior, `pip install face_recognition`, una de las principales librerías de reconocimiento facial
* OpenCV: `pip install opencv-python`, una librería muy útil para el procesamiento de imágenes
* Playsound: Primero: `pip install -U PyObjC`, Segundo: `pip install playsound`

Si se está usando **Windows**:

Siga las instrucciones anteriores excepto el uso de Anaconda para instalar dlib en el paso siguiente a la instalación de cmake : `conda install -c conda-forge dlib`

Más información para usuarios de Windows: [Presione aquí](https://stackoverflow.com/questions/41912372/dlib-installation-on-windows-10)

In [ ]:
# Para la inicialización de la cámara y la vision por computador
import cv2 
# Libreria que define tipos de datos representadas como matrices multidimensionales
import numpy as np 
# Playsound ejecuta sonidos de alerta, útiles para nuestra aplicación
from playsound import playsound
#PIL permite la edicion de imagenes desde Python
#Image provee funciones para cargar imagenes desde archivos
#ImageDraw provee funcionalidades de graficos en 2D en objetos de Image, creando nuevas imagenes, retoque y anotación de imagenes 
from PIL import Image, ImageDraw
#El modulo "face_recognition" es necesario para la detección y reconocimiento de rostros
import face_recognition
# Importar la biblioteca Keras para el trabajo de DeepLearning que puede funcionar sobre TensorFlow
from tensorflow import keras

# **Cargando el Mejor Modelo**

In [ ]:
# Asegurarse de especificar la ruta del modelo que se necesita
eye_model = keras.models.load_model('best_model_2.h5')

# **Función `eye_cropper`** 

Necesitamos esta función para optimizar la imagen para el modelo. Cuando se tome un fotograma de la cámara web, se enviará a esta función y se emitirá en un formato que el modelo sea capaz de entender, tomando en cuenta solo las características más importantes de la misma para nuestro fin.

In [ ]:
def eye_cropper(frame):

    # Crear una variable para las coordenadas de los rasgos faciales
    facial_features_list = face_recognition.face_landmarks(frame)


    # Se crea un array para marcar la posición de las coordenadas de los ojos 
    # y añadir las coordenadas de los ojos que no hayan sido encontrados 

    try:
        eye = facial_features_list[0]['left_eye']
    except:
        try:
            eye = facial_features_list[0]['right_eye']
        except:
            return


    # Establecer las coordenadas X y Y maximas de los ojos
    x_max = max([coordinate[0] for coordinate in eye])
    x_min = min([coordinate[0] for coordinate in eye])
    y_max = max([coordinate[1] for coordinate in eye])
    y_min = min([coordinate[1] for coordinate in eye])


    # Establecer el rango de las coordenadas X y Y    
    x_range = x_max - x_min
    y_range = y_max - y_min

    # Para asegurar que el ojo entero es capturado, calcular las coordenadas de un cuadrado que tenga
    # 50% de brecha al eje mayor y hacerlo coincidir el rango menor con el rango mayor
    if x_range > y_range:
        right = round(.5*x_range) + x_max
        left = x_min - round(.5*x_range)
        bottom = round((((right-left) - y_range))/2) + y_max
        top = y_min - round((((right-left) - y_range))/2)
    else:
        bottom = round(.5*y_range) + y_max
        top = y_min - round(.5*y_range)
        right = round((((bottom-top) - x_range))/2) + x_max
        left = x_min - round((((bottom-top) - x_range))/2)

    # Cortar la imagen de acuerdo a las coordenadas determinadas anteriormente
    #alls
    cropped = frame[top:(bottom + 1), left:(right + 1)]

    # Redimensionar la imagen
    cropped = cv2.resize(cropped, (80,80))
    image_for_prediction = cropped.reshape(-1, 80, 80, 3)

    return image_for_prediction

# **Script para la camara web**

* Se accede a la webcam
* Se implementa la función `eye_cropper` para optimizar cada imagen suelta para el modelo
* Pasar el fotograma recortado del ojo por el modelo para predecir si está cerrado o no
* Si la predicción da que el sujeto está cerrando los ojos durante más de 2 fotogramas, entonces lo alerta con un sonido

In [ ]:
# Iniciar la Webcam
cap = cv2.VideoCapture(0)
w = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
h = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
print(cap.get(cv2.CAP_PROP_FPS))

# Lanzar error en caso de que la camara web no se pueda abrir
if not cap.isOpened():
    raise IOError('Cannot open webcam')

# Establecer un contador
counter = 0

# Crear un bucle while que se ejecuta mientras la cámara esta en uso
while True:

    # Capturar los fotogramas que son emitidos por la cámara
    ret, frame = cap.read()

    # Utilizar cada dos fotogramas para gestionar la velocidad y el uso de la memoria
    frame_count = 0
    if frame_count == 0:
        frame_count += 1
        pass
    else:
        count = 0
        continue

    # Función llamada en el frame

    image_for_prediction = eye_cropper(frame)
    try:
        image_for_prediction = image_for_prediction/255.0
    except:
        continue

    # Obtener predicción del modelo

    prediction = eye_model.predict(image_for_prediction)

    # Basado en la predicción, mostrar si es "Ojos Abiertos" u "Ojos Cerrados"

    if prediction < 0.5:
        counter = 0
        status = 'Open'

        cv2.rectangle(frame, (round(w/2) - 110,20), (round(w/2) + 110, 80), (38,38,38), -1)

        cv2.putText(frame, status, (round(w/2)-80,70), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,255,0), 2, cv2.LINE_4)
        x1, y1,w1,h1 = 0,0,175,75
        # Dibujar rectangulos de fondo negro
        cv2.rectangle(frame, (x1,x1), (x1+w1-20, y1+h1-20), (0,0,0), -1)
        # Adicionar texto
        cv2.putText(frame, 'Active', (x1 +int(w1/10), y1+int(h1/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255,0),2)
    else:
        counter = counter + 1
        status = 'Closed'

        cv2.rectangle(frame, (round(w/2) - 110,20), (round(w/2) + 110, 80), (38,38,38), -1)

        cv2.putText(frame, status, (round(w/2)-104,70), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,255), 2, cv2.LINE_4)
        x1, y1,w1,h1 = 0,0,175,75
        # Dibujar rectangulos de fondo negro
        cv2.rectangle(frame, (x1,x1), (x1+w1-20, y1+h1-20), (0,0,0), -1)
        # Adicionar texto
        cv2.putText(frame, 'Activo', (x1 +int(w1/10), y1+int(h1/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255,0),2)

        # Si el contador es más que 3, reproducir y mostrar alerta de que el usuario esta dormido
        if counter > 2:

            x1, y1, w1, h1 = 400,400,400,100
            
            cv2.rectangle(frame, (round(w/2) - 160, round(h) - 200), (round(w/2) + 160, round(h) - 120), (0,0,255), -1)

            cv2.putText(frame, 'CONDUCTOR DURMIENDO', (round(w/2)-136,round(h) - 146), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2, cv2.LINE_4)

            cv2.imshow('Drowsiness Detection', frame)
            k = cv2.waitKey(1)
            # Emitir sonido
            playsound('rooster.mov')
            counter = 1
            continue
    cv2.imshow('Drowsiness Detection', frame)
    k = cv2.waitKey(1)
    if k == 27:
        break
cap.release()
cv2.destroyAllWindows()